In [1]:
# !pip install kaggle
# !pip install kagglehub
# 
# !pip install pandas
# !pip install numpy
# !pip install matplotlib
# !pip install Pillow
# !pip install scikit-learn
# 
# !pip install seaborn
# 
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !pip install torchsummary

# !pip install joblib

# Setup

In [2]:
# Copy from kaggle
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lantian773030/pokemonclassification")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\CindyHong\.cache\kagglehub\datasets\lantian773030\pokemonclassification\versions\1


In [3]:
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from PIL import ImageOps,ImageEnhance
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

import torch
import torch.nn.functional as F
import torch.nn as nn
from torchsummary import summary
import torchvision.transforms as transforms
from torch import optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from sklearn.metrics import confusion_matrix
from torch.utils.data import random_split
import scipy

from torchvision import models
from sklearn.utils.class_weight import compute_class_weight

from joblib import dump, load
import gc


In [4]:
np.random.seed(2024)

In [5]:
# Setting dataset path
path = os.path.join(path,'PokemonData')
path

'C:\\Users\\CindyHong\\.cache\\kagglehub\\datasets\\lantian773030\\pokemonclassification\\versions\\1\\PokemonData'

In [6]:
class_list = pd.read_csv('https://raw.githubusercontent.com/cecilia-lin/CSE151A_Pokemon_Classification/refs/heads/main/class_sheet.csv')

In [9]:
# Removing all classes with bad data
class_list = class_list[class_list['Bad_Data'] != 1]

In [10]:
# Getting Pokemons that are winged and not winged
winged = class_list[class_list['Winged'] == 1]
winged = winged['Pokemon']
not_winged = class_list[class_list['Not_Winged'] == 1]
not_winged = not_winged['Pokemon']

In [11]:
# Making a data frame that has the image paths and their classifcation
images = []
wrong_extensions = []

for folder in winged:
    image_names = os.listdir(os.path.join(path,folder))
    for image in image_names:
        file_extension = os.path.splitext(image)[1]
        # Sanity check to make sure all file paths exist
        if file_extension not in valid_extensions:
            wrong_extensions.append(file_extension)
            continue
        image_path = os.path.join(path,folder,image)
        if not os.path.exists(image_path):
            print(image_path)
            continue
        with Image.open(image_path) as img:
            img = np.array(img)
            images.append({'Image': img, 'Source': 'Winged'})
            
for folder in not_winged:
    image_names = os.listdir(os.path.join(path,folder))
    for image in image_names:
        file_extension = os.path.splitext(image)[1]
        # Sanity check to make sure all file paths exist
        if file_extension not in valid_extensions:
            wrong_extensions.append(file_extension)
            continue
        image_path = os.path.join(path,folder,image)
        if not os.path.exists(image_path):
            print(image_path)
            continue
        with Image.open(image_path) as img:
            img = np.array(img)
            images.append({'Image': img, 'Source': 'Not Winged'})
            
df = pd.DataFrame(images)
set(wrong_extensions)

{'.asp',
 ".jpg')",
 '.jpg~c200',
 '.sb-334870d9-FK2TVI',
 '.sb-334870d9-nlD60H',
 '.svg'}

In [12]:
# Sanity check to make sure there are only 2 classes
for index, row in df.iterrows():
    if row['Source'] not in ['Winged', 'Not Winged']:
        print(row)

# Data Preprocessing

In [19]:
# Normalizing function
def zscore(img,default = False):
    '''
    Apply Z-score normalization to an image.

    Parameters:
    - img (numpy array): The input image array.
    - default (bool): Whether to use default ImageNet mean and std.

    Returns:
    - img (numpy array): The normalized image array.
    '''
    img = img.astype(np.float32)
    for c in range(img.shape[2]):
        if not default:
            mean = img[:,:,c].mean()
            std = img[:,:,c].std()
        else:
            # Use default mean and std from ImageNet
            mean=[0.485, 0.456, 0.406][c]
            std=[0.229, 0.224, 0.225][c]
        if std > 0:
            img[:,:,c] = (img[:,:,c] - mean) / std
        else:
            img[:,:,c] = (img[:,:,c] - mean)
    return img

def minmax(img):
    '''
    Apply Min-Max normalization to an image.

    Parameters:
    - img (numpy array): The input image array.

    Returns:
    - img (numpy array): The normalized image array.
    '''
    img = img.astype(np.float32)
    for c in range(img.shape[2]):
        max = img[:,:,c].max()
        min = img[:,:,c].min()
        img[:,:,c] = (img[:,:,c] - min) / (max - min)
    return img

In [20]:
# Function to preprocess images
def preprocess(df, pattern, default=False, batch_size=50):
    '''
    Preprocess images by resizing, converting to grayscale or RGB, and normalizing pixel values.

    Parameters:
    - df (DataFrame): DataFrame containing images and labels.
    - pattern (str): Normalization pattern ('Zscore', 'Minmax', 'Grayscale').
    - default (bool): Whether to use default mean and std for Z-score normalization.
    - batch_size (int): Number of images to process in each batch.

    Returns:
    - X (numpy array): Array of preprocessed images.
    - y (numpy array): Array of corresponding labels.
    '''

    X = []
    y = []
    new_size = (224, 224)

     # Process images in batches to manage memory usage
    for start in range(0, len(df), batch_size):
        batch_df = df.iloc[start:start + batch_size]
        X_batch = []
        y_batch = []
        for _, row in batch_df.iterrows():
            # Resize images to 224 x 224
            img = Image.fromarray(row['Image']).resize(new_size)
            if pattern == 'Grayscale':
                # Convert images to grayscale
                img = img.convert('L')
                img = np.array(img) / 255.0
            else:
                img = img.convert('RGB')
                img = np.array(img)
                # Zscore normalization
                if pattern == 'Zscore':
                    img = zscore(img, default=default)
                # Minmax normalization
                elif pattern == 'Minmax':
                    img = minmax(img)

            X_batch.append(img)
            y_batch.append(1 if row['Source'] == 'Winged' else 0)
        
        X.append(np.array(X_batch))
        y.append(np.array(y_batch))
        
        # Release memory
        del X_batch, y_batch
        gc.collect()
    
    # Concatenate batches into full arrays
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    return X, y # X shape: (n_samples, 224, 224, 3) or (n_samples, 224, 224)

In [27]:
######## We can also use some optimization methods to train on imbalanced dataset ########
# Balancing the data set by random sampling
def random_sample(X,y,size=600):
    '''
    Balance the dataset by randomly sampling equal numbers of images from each class.

    Parameters:
    - X (numpy array): Array of images.
    - y (numpy array): Array of labels.

    Returns:
    - X (numpy array): Balanced array of images.
    - y (numpy array): Balanced array of labels.
    '''
    id_not_winged = np.where(y == 0)[0]
    id_winged = np.where(y == 1)[0]
    sample_not_winged = np.random.choice(id_not_winged, size, replace=False)
    sample_winged = np.random.choice(id_winged, size, replace=False)
    sample = np.concatenate((sample_not_winged, sample_winged), axis=0)
    X = X[sample]
    y = y[sample]
    return X,y
######## We can also use some optimization methods to train on imbalanced dataset ########

# Creating Train/Validate/Test splits (80/10/10)
def random_split(X,y,train_size=0.8):
    '''
    Split the data into training, validation, and test sets (80% train, 10% val, 10% test).

    Parameters:
    - X (numpy array): Array of images.
    - y (numpy array): Array of labels.

    Returns:
    - X_train, X_val, X_test: Split arrays of images.
    - y_train, y_val, y_test: Split arrays of labels.
    '''
    #Split into training and temporary sets
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=1-train_size, random_state=2024, stratify=y)

    #Split the temporary set into validation and test sets
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=2024, stratify=y_temp)
    return X_train, X_val, X_test, y_train, y_val,y_test

# X , y = random_sample(X,y)
# X_train, X_val, X_test, y_train, y_val,y_test = random_split(X,y)

In [28]:
# Augmentation function
# Our resized image doesn't have enough space for shearing or zooming
def augmentation(img,pattern,img_size=(224,224),horizontal_flip=True,vertical_flip=False,rotation_range=20, horizontal_shift=0.2, vertical_shift=0.2):
    '''
    Apply data augmentation techniques to a single image.

    Parameters:
    - img (numpy array): Input image.
    - pattern (str): Normalization pattern used ('Zscore', 'Minmax', 'Grayscale').
    - img_size (tuple): Desired output image size.
    - horizontal_flip (bool): Whether to apply horizontal flip.
    - vertical_flip (bool): Whether to apply vertical flip.
    - rotation_range (int): Range of degrees for random rotations.
    - horizontal_shift (float): Fraction of width for horizontal shift.
    - vertical_shift (float): Fraction of height for vertical shift.

    Returns:
    - img (numpy array): Augmented image.
    '''
    if img.dtype == np.float32:
        # Convert image back to uint8 for PIL processing
        img = (img * 255).astype(np.uint8)
    if pattern != 'Grayscale':
        # Calculate mean pixel value for fill color
        mean = tuple(np.mean(img,axis=(0,1)).astype(int))
    else:
        mean = 0
    if isinstance(img, np.ndarray):
        img = Image.fromarray(img)

    # Apply horizontal flip with 50% probability
    if horizontal_flip and np.random.random() < 0.5:
        img = img.transpose(Image.Transpose.FLIP_LEFT_RIGHT)

    # Apply vertical flip with 50% probability
    if vertical_flip and np.random.random() < 0.5:
        img = img.transpose(Image.Transpose.FLIP_TOP_BOTTOM)

    # Apply random rotation within the specified range
    if rotation_range > 0:
        rotation_range = np.random.randint(-rotation_range, rotation_range)
        img = img.rotate(rotation_range, resample=Image.BICUBIC,fillcolor=mean)

    # Apply random shifts
    if horizontal_shift > 0 or vertical_shift > 0:
        w , h = img_size
        dw = np.random.randint(-horizontal_shift * w, horizontal_shift * w)
        dh = np.random.randint(-vertical_shift * h, vertical_shift * h)
        img = ImageOps.expand(img, border=(abs(dw),abs(dh),abs(dw),abs(dh)),fill=mean)
        img = img.crop((dw+abs(dw),dh+abs(dh),dw+abs(dw)+w,dh+abs(dh)+h))
    # Ensure the image is resized back to the original dimensions
    img = img.resize(img_size)

    # Convert image back to numpy array and normalize pixel values
    img = np.array(img).astype(np.float32) / 255.0
    return img

In [29]:
# Going through all images and augment them
def augment_loop(X_train,y_train,pattern,img_size=(224,224),horizontal_flip=True,vertical_flip=False,rotation_range=20, horizontal_shift=0.2, vertical_shift=0.2):
    '''
    Apply augmentation to the training dataset.

    Parameters:
    - X_train (numpy array): Training images.
    - y_train (numpy array): Training labels.
    - pattern (str): Normalization pattern used.
    - img_size (tuple): Desired output image size.
    - Other parameters: Augmentation options.

    Returns:
    - X_train (numpy array): Augmented training images.
    - y_train (numpy array): Corresponding labels.
    '''
    
    X_augmented = []
    y_augmented = []
    for img, label in zip(X_train,y_train):
        X_augmented.append(img) # Original image
        img = augmentation(img,pattern,img_size,horizontal_flip,vertical_flip,rotation_range, horizontal_shift, vertical_shift)
        X_augmented.append(img) # Augmented image
        y_augmented.append(label)
        y_augmented.append(label)
    del img, label
    gc.collect()
    X_train = np.array(X_augmented)
    y_train = np.array(y_augmented)
    return X_train, y_train

# Model Training

## DL

### Data Visualization

In [ ]:
def calculate_accuracy(loader, model, device):
    model.eval() 
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

def get_losses(losses,loader, model, device):
    loss = 0.0
    model.eval() 
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            l = criterion(outputs, targets)
            loss += l.item()
    loss /= len(loader)
    losses.append(loss)
    return losses

def get_predictions(loader, model, device):
    all_targets = []
    all_predictions = []
    model.eval() 
    # for epoch in range(epoches):
    with torch.no_grad():
        for i, (inputs, targets) in enumerate(loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_targets.extend(targets.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
    return all_targets,all_predictions

# Plot loss curve and confusion matrix for NNs
def loss_curve(epoches,train_losses,val_losses,model = ''):
    folder_path = os.path.join('results',model)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    # loss curve
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, epoches + 1), train_losses, label='Train Loss')
    plt.plot(range(1, epoches + 1), val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training and Testing Loss Curve')
    plt.legend()
    plt.savefig(os.path.join(folder_path,'Loss_Curve.png'))
    plt.show()

def plot_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = ''):
    folder_path = os.path.join('results',model)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        
    plt.figure(figsize=(18, 5))
    plt.subplot(1, 3, 1)
    # calculate confusion matrix
    cm = confusion_matrix(all_train_targets, all_train_predictions)
    # plot confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix - Train')
    
    plt.subplot(1, 3, 2)
    # calculate confusion matrix
    cm = confusion_matrix(all_val_targets, all_val_predictions)
    # plot confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix - Val')

    
    plt.subplot(1, 3, 3)
    # calculate confusion matrix
    cm = confusion_matrix(all_test_targets, all_test_predictions)
    # plot confusion matrix
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix - Test')
    
    plt.savefig(os.path.join(folder_path,'Confusion_Matrix.png'))
    plt.show()
    
def print_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = ''):
    folder_path = os.path.join('results',model)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    report_train = classification_report(all_train_targets,all_train_predictions)
    report_val = classification_report(all_val_targets,all_val_predictions)
    report_test = classification_report(all_test_targets,all_test_predictions)
    with open(os.path.join(folder_path,'report.txt'), 'w') as f:
        f.write("\nreport_train\n")
        f.write(report_train)
        f.write("\nreport_val\n")
        f.write(report_val)
        f.write("\nreport_test\n")
        f.write(report_test)

### Dataloader

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = torch.tensor(self.data[idx],dtype=torch.float32)
        label = torch.tensor(self.labels[idx],dtype=torch.long)
        
        return image, label

### CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self,image_size=(3,224,224), num_classes=2,dropout = 0.25):
        super(CNN, self).__init__()
        c,h,w = image_size
        self.conv1 = nn.Conv2d(3, 64, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.bn2 = nn.BatchNorm2d(128)
        # self.bn3 = nn.BatchNorm2d(256)
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout)
        self.flattened_size = 128 * (h // 4) * (w // 4) 
        self.fc = nn.Linear(self.flattened_size, num_classes)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        # x = self.bn3(self.conv3(x))
        # x = self.pool(x)
        
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
pattern = 'Minmax'
X,y = preprocess(df,pattern)
X , y = random_sample(X,y)
X_train, X_val, X_test, y_train, y_val,y_test = random_split(X,y)
X_train, y_train = augment_loop(X_train,y_train,pattern)

In [ ]:
X_train = X_train.transpose(0,3,1,2)
X_val = X_val.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)
X_train.shape

In [ ]:
torch.manual_seed(2024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

CNN = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(CNN.parameters(),lr=0.01,weight_decay=1e-4)

epoches = 50
best_val_accuracy = 0
datas = []
val_datas = []

# Create DataLoaders for training and testing
dataset = ImageDataset(X_train,y_train)
val_dataset = ImageDataset(X_val,y_val)
test_dataset = ImageDataset(X_test,y_test)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

train_losses = []
val_losses = []

for epoch in range(epoches):
    CNN.train()
       
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = CNN(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    
    train_accuracy = calculate_accuracy(train_loader, CNN, device)
    val_accuracy = calculate_accuracy(val_loader, CNN, device)
    
    print(f"Epoch [{epoch + 1}/{epoches}], "
          f"Loss: {loss.item() / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {val_accuracy:.2f}%")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(CNN.state_dict(), 'best_model_CNN.pth')
    
    train_losses = get_losses(train_losses,train_loader, CNN, device)
    val_losses = get_losses(val_losses,val_loader, CNN, device)



In [ ]:
# evaluate model
CNN.load_state_dict (torch.load('best_model_CNN.pth'))

all_train_targets = []
all_train_predictions = []

all_train_targets,all_train_predictions = get_predictions(train_loader, CNN, device)

all_val_targets = []
all_val_predictions = []

all_val_targets,all_val_predictions = get_predictions(val_loader, CNN, device)

all_test_targets = []
all_test_predictions = []

all_test_targets,all_test_predictions = get_predictions(test_loader, CNN, device)

# loss curve
loss_curve(epoches, train_losses, val_losses,model = 'CNN')

plot_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = 'CNN')



In [ ]:
print_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = 'CNN')

torch.cuda.empty_cache()

### Resnet18

In [ ]:
pattern = 'Zscore'
X,y = preprocess(df,pattern,default=True)
# X , y = random_sample(X,y)
X_train, X_val, X_test, y_train, y_val,y_test = random_split(X,y)
# X_train, y_train = augment_loop(X_train,y_train,pattern)

In [ ]:
X_train = X_train.transpose(0,3,1,2)
X_val = X_val.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)
X_train.shape

In [ ]:
torch.manual_seed(2024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Resnet18 = models.resnet18(pretrained=True)
Resnet18.fc = nn.Linear(Resnet18.fc.in_features, 2)  # Binary classification (0 and 1)
Resnet18 = Resnet18.to(device)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
optimizer = optim.Adam(Resnet18.parameters(), lr=0.001, weight_decay=1e-4)

epoches = 50
best_val_accuracy = 0
datas = []
val_datas = []

# Create DataLoaders for training and testing
dataset = ImageDataset(X_train,y_train)
val_dataset = ImageDataset(X_val,y_val)
test_dataset = ImageDataset(X_test,y_test)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
   
train_losses = []
val_losses = []

for epoch in range(epoches):
    Resnet18.train()   
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device).long()
        optimizer.zero_grad()
        outputs = Resnet18(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    train_accuracy = calculate_accuracy(train_loader, Resnet18, device)
    val_accuracy = calculate_accuracy(val_loader, Resnet18, device)
    
    print(f"Epoch [{epoch + 1}/{epoches}], "
          f"Loss: {loss.item() / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {val_accuracy:.2f}%")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(Resnet18.state_dict(), 'best_model_resnet18.pth')
    
    train_losses = get_losses(train_losses,train_loader, Resnet18, device)
    val_losses = get_losses(val_losses,val_loader, Resnet18, device)


In [ ]:
# evaluate model
Resnet18.load_state_dict (torch.load('best_model_resnet18.pth'))

all_train_targets = []
all_train_predictions = []

all_train_targets,all_train_predictions = get_predictions(train_loader, Resnet18, device)

all_val_targets = []
all_val_predictions = []

all_val_targets,all_val_predictions = get_predictions(val_loader, Resnet18, device)

all_test_targets = []
all_test_predictions = []

all_test_targets,all_test_predictions = get_predictions(test_loader, Resnet18, device)

# loss curve
loss_curve(epoches, train_losses, val_losses,model = 'Resnet18')

plot_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = 'Resnet18')



In [ ]:
print_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = 'Resnet18')

torch.cuda.empty_cache()

In [ ]:
pattern = 'Zscore'
X,y = preprocess(df,pattern,default=True)
# X , y = random_sample(X,y)
X_train, X_val, X_test, y_train, y_val,y_test = random_split(X,y)
X_train, y_train = augment_loop(X_train,y_train,pattern)

In [ ]:
X_train = X_train.transpose(0,3,1,2)
X_val = X_val.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)
X_train.shape

In [ ]:
torch.manual_seed(2024)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

Resnet18 = models.resnet18(pretrained=True)
Resnet18.fc = nn.Linear(Resnet18.fc.in_features, 2)  # Binary classification (0 and 1)
Resnet18 = Resnet18.to(device)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))
optimizer = optim.Adam(Resnet18.parameters(), lr=0.001, weight_decay=1e-4)

epoches = 50
best_val_accuracy = 0
datas = []
val_datas = []

# Create DataLoaders for training and testing
dataset = ImageDataset(X_train,y_train)
val_dataset = ImageDataset(X_val,y_val)
test_dataset = ImageDataset(X_test,y_test)
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
   
train_losses = []
val_losses = []

for epoch in range(epoches):
    Resnet18.train()   
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device).long()
        optimizer.zero_grad()
        outputs = Resnet18(inputs)

        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    train_accuracy = calculate_accuracy(train_loader, Resnet18, device)
    val_accuracy = calculate_accuracy(val_loader, Resnet18, device)
    
    print(f"Epoch [{epoch + 1}/{epoches}], "
          f"Loss: {loss.item() / len(train_loader):.4f}, "
          f"Train Accuracy: {train_accuracy:.2f}%, "
          f"Test Accuracy: {val_accuracy:.2f}%")
    
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(Resnet18.state_dict(), 'best_model_resnet18_aug.pth')
    
    train_losses = get_losses(train_losses,train_loader, Resnet18, device)
    val_losses = get_losses(val_losses,val_loader, Resnet18, device)


In [ ]:
# evaluate model
Resnet18.load_state_dict (torch.load('best_model_resnet18_aug.pth'))

all_train_targets = []
all_train_predictions = []

all_train_targets,all_train_predictions = get_predictions(train_loader, Resnet18, device)

all_val_targets = []
all_val_predictions = []

all_val_targets,all_val_predictions = get_predictions(val_loader, Resnet18, device)

all_test_targets = []
all_test_predictions = []

all_test_targets,all_test_predictions = get_predictions(test_loader, Resnet18, device)

# loss curve
loss_curve(epoches, train_losses, val_losses,model = 'Resnet18_aug')

plot_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = 'Resnet18_aug')



In [ ]:
print_confusion_matrix(all_train_targets,all_train_predictions,all_val_targets,all_val_predictions,all_test_targets,all_test_predictions,model = 'Resnet18_aug')

torch.cuda.empty_cache()